In [1]:
print("hello")

hello


In [2]:
%pip install chromadb sentence-transformers openai python-dotenv pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### 🪜 Step 1 — Imports & Environment Setup

In [29]:
import os
import pandas as pd
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
from transformers import pipeline

In [30]:
# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [ ]:
VECTOR_DB_PATH = "C:/Users/shiva/OneDrive/Documents/Material/Amazon_Ecommorce_agent/data/vectorstore/"
COLLECTION_NAME = "products"
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"


### 🧩 Step 2 — Connect to Chroma & Embedding Function

In [33]:
from chromadb.utils import embedding_functions

embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBED_MODEL
)

# Connect to persistent Chroma vectorstore
client_chroma = chromadb.PersistentClient(path=VECTOR_DB_PATH)

# Load the existing collection
collection = client_chroma.get_or_create_collection(
    name=COLLECTION_NAME,
    
)

print("✅ Connected to existing Chroma vectorstore")


✅ Connected to existing Chroma vectorstore


### 🔍 Step 3 — Define a Retrieval Function

In [34]:
def retrieve_products(query, n_results=5):
    """Retrieve top products for a query and return DataFrame + raw results."""
    results = collection.query(query_texts=[query], n_results=n_results)

    # Extract all metadata and distances properly
    metadatas = results["metadatas"][0]
    distances = results["distances"][0]

    # Convert to DataFrame
    df = pd.DataFrame(metadatas)

    # Add similarity score
    df["similarity"] = distances

    # Sort by similarity (lower = closer in embedding space)
    df = df.sort_values(by="similarity", ascending=True).reset_index(drop=True)

    return df, results


###  Step 4 — Build RAG Pipeline Function

In [20]:
# --- Step 4: Load lightweight local LLM ---
def get_local_llm():
    """Load a small, CPU-friendly model."""
    model_name = "google/flan-t5-base"   # < 1 GB RAM on CPU
    print(f"🤖 Loading local model: {model_name}")
    return pipeline("text2text-generation", model=model_name, device=-1)

llm = get_local_llm()


🤖 Loading local model: google/flan-t5-base


config.json: 0.00B [00:00, ?B/s]

c:\Users\shiva\OneDrive\Documents\Material\Amazon_Ecommorce_agent\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shiva\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [53]:
def generate_rag_response(query, top_k=5):
    """Combine retrieval and generation for product recommendations."""
    retrieved_df, results = retrieve_products(query, n_results=top_k)
    if retrieved_df is None or results is None:
        return None, "⚠️ No relevant products found."

    # 🧠 Build concise product context
    context = "\n".join([
        f"{item.get('name','N/A')} — ₹{item.get('discount_price','N/A')} | ⭐ {item.get('ratings','N/A')} | {item.get('sub_category','')}"
        for item in results["metadatas"][0]
    ])

    # 🗣️ Strongly worded generation prompt
    prompt = f"""
You are an expert e-commerce product recommendation assistant.
The user is searching for: "{query}"

Here are some matching products:
{context}

Now write the final response in this format:
- Product Name — ₹<price> — ⭐ <rating> — <short reason in ≤ 12 words>

After listing up to {top_k} products, write 1–2 sentences recommending the single best one overall.

Rules:
- Do NOT repeat these instructions.
- Do NOT include JSON, code, or markdown.
- Write plain text only.
""".strip()

    # 🧩 Generate response
    output = llm(prompt, max_new_tokens=256)[0]["generated_text"].strip()

    return retrieved_df, output


### 💬 Step 5 — Test a Query

In [54]:
# --- Step 6: Test Query ---
query = "Best budget smartphone under ₹15000 with good camera"

retrieved_df, answer = generate_rag_response(query)

print("📦 Retrieved Products:")

# 🔧 Ensure DataFrame has the required columns for display
# If columns differ, rename them accordingly based on your metadata schema
# Example: if your metadata used 'product_name' instead of 'name', rename it here

column_map = {
    "product_name": "name",
    "discount": "discount_price",
    "rating": "ratings",
    "category": "sub_category"
}

# Rename if keys exist
retrieved_df = retrieved_df.rename(columns={k: v for k, v in column_map.items() if k in retrieved_df.columns})

# Now safely display
display(retrieved_df[["name", "discount_price", "ratings", "sub_category"]].head())

print("\n🤖 AI Assistant Response:\n")
print(answer)


📦 Retrieved Products:


,name,discount_price,ratings,sub_category
0,"Mobile Phone External Camera, USB Camera Modul...",2234.5,3.3,Security Cameras
1,"Great Choice WiFi Camera, HD 1080P Wireless Wi...",1549.0,3.3,Security Cameras
2,"Nokia G11 Android 12 Smartphone, Dual SIM, 3-D...",9999.0,3.9,All Electronics
3,Great Choice Full HD 1440p Portable Home Secur...,1400.0,2.0,Security Cameras
4,"Nokia G11 Android 12 Smartphone, Dual SIM, 3-D...",9999.0,3.9,All Electronics



🤖 AI Assistant Response:

Great Choice Full HD 1440p Portable Home Security USB Charger Adapter Camera,SG CCTV with Night Vision, Motion Detection — 9999.0 |  3.9 | All Electronics Great Choice Full HD 1440p Portable Home Security USB Charger Adapter Camera,SG CCTV with Night Vision, Motion Detection — 1400.0 |  2.0 | Security Cameras Nokia G11 Android 12 Smartphone, Dual SIM, 3-Day Battery Life, 4GB RAM + 64GB Storage, 50MP Dual AI Camera | Lake Blue — 9999.0 |  3.9 | All Electronics Great Choice Full HD 1440p Portable Home Security USB Charger Adapter Camera,SG CCTV with Night Vision, Motion Detection — 1400.0 |  2.0 | Security Cameras Nokia G11 Android 12 Smartphone, Dual SIM, 3-Day Battery Life, 4GB RAM + 64GB Storage, 50MP Dual AI Camera | Charcoal Grey — 9999.0 |  3.9 | All Electronics Great Choice Full HD 1440p Portable Home Security USB Charger Adapter Camera,SG CCTV with Night Vision, Motion Detection


In [61]:
# Save your working generate_rag_response function to a Python file
import inspect

with open("rag_agent.py", "w", encoding="utf-8") as f:
    f.write(inspect.getsource(generate_rag_response))
    print("✅ Function exported to rag_agent.py")


✅ Function exported to rag_agent.py
